In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, mean_squared_error
import ast
import xgboost as xgb
import ast
from sklearn.preprocessing import MinMaxScaler


In [23]:
data=pd.read_csv('Hotel_Reviews_2.csv') #read the csv file
print(data.head(40))

                                        Hotel_Address  \
0    s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
1    s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
2    s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
3    s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
4    s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
5    s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
6    s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
7    s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
8    s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
9    s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
10   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
11   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
12   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
13   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
14   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
15   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
16   s Gravesandestraat 55 Oost

In [24]:
# This is the basic data cleaning process. This should be added in all the models
# This cell is a bit slow to run, so it is intended to be run only once

# This part drops the columns that are not needed in the model
data.drop('Hotel_Address',axis=1,inplace=True)  # Drop the column 'Hotel_Address' due to lack of relevance
data.drop('Review_Date',axis=1,inplace=True)    # Drop the column 'Review_Date' due to lack of complete data
data.drop('Additional_Number_of_Scoring',axis=1,inplace=True)  # Drop the column 'Additional_Number_of_Scoring' due to not knowing what the numbers mean
data.drop('lat',axis=1,inplace=True)    # Drop the column 'lat' due to lack of relevance
data.drop('lng',axis=1,inplace=True)    # Drop the column 'lng' due to lack of relevance
data.drop('Total_Number_of_Reviews',axis=1,inplace=True)    # Drop the column 'Total_Number_of_Reviews' due to the number seams to be incorrect


# The next part of the code aims to split the 'Tags' column into multiple columns

data['Tags'] = data['Tags'].apply(ast.literal_eval) # Convert the string to a list, This code is made by chatgpt

# The following 3 lines of code is made by github copilot

tags_expanded = data['Tags'].apply(pd.Series) # Expand the 'Tags' column into multiple columns

tags_expanded.columns = [f'Tag_{i}' for i in range(tags_expanded.shape[1])] # Rename the columns for better readability

data = pd.concat([data, tags_expanded], axis=1) # Concatenate the expanded tags with the original dataframe

data.drop('Tags',axis=1,inplace=True)   # Drop the column 'Tags' due to the data being split into multiple columns

# The next part of the code turns the 'days_since_review' and 'Tag_3' columns into integers
data['days_since_review']=data['days_since_review'].str.extract('(\d+)').astype(int) # Extract the number from the string. This code if made using chatgpt
data['Tag_3'] = data['Tag_3'].str.extract('(\d+)').astype(float) # Extract the number from the string and convert to float. For some reason it did not work as int

# The next part of the code creates a new dataframe with hotel names and removes it from the dataframe data
hotel_names = data['Hotel_Name'] # Create a new dataframe with the hotel names
data.drop('Hotel_Name',axis=1,inplace=True) # Drop the column 'Hotel_Name' as it is not needed in the model

data.dropna(inplace=True) # Drop the NaN in the colum

In [25]:
print(data.head())

       Average_Score    Reviewer_Nationality  \
20097            9.1   United Arab Emirates    
40699            8.4         United Kingdom    
48884            8.8               Portugal    
54890            8.5         United Kingdom    
71335            8.1         United Kingdom    

                                         Negative_Review  \
20097   My only complaint would be that dogs are welc...   
40699   No bar or restaurant onsite Better facilities...   
48884                                            Nothing   
54890                                        No Negative   
71335   Not much to complain about we shopped around ...   

       Review_Total_Negative_Word_Counts  \
20097                                 69   
40699                                 20   
48884                                  2   
54890                                  0   
71335                                 52   

                                         Positive_Review  \
20097   Beautiful rooms l

In [26]:
knn_data = data.copy()

knn_data.drop(['Reviewer_Nationality', 'Negative_Review', 'Positive_Review'], axis=1, inplace=True)

# One hot encoding
knn_data = pd.get_dummies(knn_data, dtype=int)

print(knn_data.head())






       Average_Score  Review_Total_Negative_Word_Counts  \
20097            9.1                                 69   
40699            8.4                                 20   
48884            8.8                                  2   
54890            8.5                                  0   
71335            8.1                                 52   

       Review_Total_Positive_Word_Counts  \
20097                                 28   
40699                                 29   
48884                                 75   
54890                                  2   
71335                                 48   

       Total_Number_of_Reviews_Reviewer_Has_Given  Reviewer_Score  \
20097                                          21             9.2   
40699                                           2             8.3   
48884                                          28            10.0   
54890                                          11             9.2   
71335                              

In [32]:

percentage = 1
sample_size = int(knn_data.shape[0] * percentage)
knn_data_sample = knn_data.sample(n=sample_size, random_state=69)

features = knn_data_sample.columns.difference(['Average_Score'])
scaler = MinMaxScaler()
knn_data_sample[features] = scaler.fit_transform(knn_data_sample[features])

X = knn_data_sample.drop('Average_Score', axis=1) 
y = knn_data_sample['Average_Score'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print(f'KNN Modellens Nøyaktighet: {mse:.4f}')

KNN Modellens Nøyaktighet: 0.1805
